In [33]:
# Kết nối Google Drive (nếu chạy trong Colab)
try:
    from google.colab import drive
    print("Kết nối Google Drive...")
    drive.mount('/content/drive')
except:
    print("Không thể kết nối Google Drive hoặc không chạy trong môi trường Colab.")

Kết nối Google Drive...
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# **explore_data_detailed**

In [34]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.io as sio
import os
from scipy.fft import fft, fftfreq
import pandas as pd

# Đường dẫn đến file dữ liệu
data_path = '/content/drive/MyDrive/bidmc/bidmc-ppg-and-respiration-dataset-1.0.0/bidmc_data.mat'
figures_path = '/content/drive/MyDrive/bidmc/figures'
os.makedirs(figures_path, exist_ok=True)

# Tải dữ liệu
print("Đang tải dữ liệu từ file .mat...")
mat_data = sio.loadmat(data_path)
data = mat_data['data'][0]  # Lấy mảng chính chứa 53 bản ghi

# Khám phá cấu trúc dữ liệu
print(f"Số lượng bản ghi: {len(data)}")

# Khám phá cấu trúc chi tiết của bản ghi đầu tiên
first_record = data[0]
print("\nKhám phá cấu trúc chi tiết của bản ghi đầu tiên:")

# Kiểm tra cấu trúc của trường ppg
ppg_field = first_record['ppg'][0, 0]
print(f"Cấu trúc của trường ppg: {type(ppg_field)}")
if hasattr(ppg_field, 'dtype') and hasattr(ppg_field.dtype, 'names'):
    print(f"Các trường con của ppg: {ppg_field.dtype.names}")

# Kiểm tra cấu trúc của trường ref
ref_field = first_record['ref'][0, 0]
print(f"Cấu trúc của trường ref: {type(ref_field)}")
if hasattr(ref_field, 'dtype') and hasattr(ref_field.dtype, 'names'):
    print(f"Các trường con của ref: {ref_field.dtype.names}")

    # Kiểm tra cấu trúc của trường params trong ref
    if 'params' in ref_field.dtype.names:
        params_field = ref_field['params'][0, 0]
        print(f"Cấu trúc của trường params: {type(params_field)}")
        if hasattr(params_field, 'dtype') and hasattr(params_field.dtype, 'names'):
            print(f"Các trường con của params: {params_field.dtype.names}")

# Kiểm tra chi tiết hơn về cấu trúc dữ liệu
print("\nKiểm tra chi tiết hơn về cấu trúc dữ liệu:")
print(f"Kiểu dữ liệu của ppg.v: {type(first_record['ppg'][0, 0]['v'])}")
print(f"Kích thước của ppg.v: {first_record['ppg'][0, 0]['v'].shape}")

# Kiểm tra cấu trúc của HR và RR
print("\nKiểm tra cấu trúc của HR và RR:")
hr_field = first_record['ref'][0, 0]['params'][0, 0]['hr'][0]
print(f"Kiểu dữ liệu của hr: {type(hr_field)}")
print(f"Kích thước của hr: {hr_field.shape}")
print(f"Giá trị đầu tiên của hr: {hr_field[0] if hr_field.size > 0 else 'Không có dữ liệu'}")

rr_field = first_record['ref'][0, 0]['params'][0, 0]['rr'][0]
print(f"Kiểu dữ liệu của rr: {type(rr_field)}")
print(f"Kích thước của rr: {rr_field.shape}")
print(f"Giá trị đầu tiên của rr: {rr_field[0] if rr_field.size > 0 else 'Không có dữ liệu'}")

# Trích xuất và vẽ tín hiệu PPG từ bản ghi đầu tiên
try:
    # Truy cập trực tiếp vào dữ liệu PPG
    ppg_data = first_record['ppg'][0, 0]['v']
    if isinstance(ppg_data, np.ndarray):
        # Nếu là mảng numpy, lấy dữ liệu trực tiếp
        sample_ppg = ppg_data.flatten()
    else:
        # Nếu không phải mảng numpy, chuyển đổi thành mảng
        sample_ppg = np.array(ppg_data, dtype=float).flatten()

    # Lấy tần số lấy mẫu
    sample_fs_ppg = float(first_record['ppg'][0, 0]['fs'][0, 0])

    # Tương tự cho ECG và Resp
    ecg_data = first_record['ekg'][0, 0]['v']
    if isinstance(ecg_data, np.ndarray):
        sample_ecg = ecg_data.flatten()
    else:
        sample_ecg = np.array(ecg_data, dtype=float).flatten()

    sample_fs_ecg = float(first_record['ekg'][0, 0]['fs'][0, 0])

    resp_data = first_record['ref'][0, 0]['resp_sig'][0, 0]['imp'][0, 0]['v']
    if isinstance(resp_data, np.ndarray):
        sample_resp = resp_data.flatten()
    else:
        sample_resp = np.array(resp_data, dtype=float).flatten()

    sample_fs_resp = float(first_record['ref'][0, 0]['resp_sig'][0, 0]['imp'][0, 0]['fs'][0, 0])

    print(f"\nTần số lấy mẫu PPG: {sample_fs_ppg} Hz")
    print(f"Tần số lấy mẫu ECG: {sample_fs_ecg} Hz")
    print(f"Tần số lấy mẫu Resp: {sample_fs_resp} Hz")

    print(f"Độ dài tín hiệu PPG: {len(sample_ppg)} mẫu")
    print(f"Độ dài tín hiệu ECG: {len(sample_ecg)} mẫu")
    print(f"Độ dài tín hiệu Resp: {len(sample_resp)} mẫu")

    # Tính thời gian cho trục x
    time_ppg = np.arange(len(sample_ppg)) / sample_fs_ppg
    time_ecg = np.arange(len(sample_ecg)) / sample_fs_ecg
    time_resp = np.arange(len(sample_resp)) / sample_fs_resp

    # Vẽ tín hiệu mẫu
    plt.figure(figsize=(15, 10))

    plt.subplot(3, 1, 1)
    plt.plot(time_ecg[:1000], sample_ecg[:1000])
    plt.title('ECG Signal (First Record - First 1000 samples)')
    plt.xlabel('Time (s)')
    plt.ylabel('Amplitude')

    plt.subplot(3, 1, 2)
    plt.plot(time_ppg[:1000], sample_ppg[:1000])
    plt.title('PPG Signal (First Record - First 1000 samples)')
    plt.xlabel('Time (s)')
    plt.ylabel('Amplitude')

    plt.subplot(3, 1, 3)
    plt.plot(time_resp[:1000], sample_resp[:1000])
    plt.title('Respiratory Signal (First Record - First 1000 samples)')
    plt.xlabel('Time (s)')
    plt.ylabel('Amplitude')

    plt.tight_layout()
    plt.savefig(os.path.join(figures_path, 'sample_signals.png'))
    plt.close()

    # Phân tích phổ tần số của tín hiệu PPG
    def plot_fft(signal, fs, title, filename):
        N = len(signal)
        T = 1.0 / fs
        yf = fft(signal)
        xf = fftfreq(N, T)[:N//2]

        plt.figure(figsize=(10, 6))
        plt.plot(xf, 2.0/N * np.abs(yf[0:N//2]))
        plt.grid(True, alpha=0.3)
        plt.title(f'FFT of {title}')
        plt.xlabel('Frequency (Hz)')
        plt.ylabel('Amplitude')
        plt.xlim(0, 5)  # Giới hạn tần số hiển thị đến 5Hz
        plt.savefig(os.path.join(figures_path, filename))
        plt.close()

    # Phân tích phổ tần số của tín hiệu PPG, ECG và Resp
    plot_fft(sample_ppg, sample_fs_ppg, 'PPG Signal', 'ppg_fft.png')
    plot_fft(sample_ecg, sample_fs_ecg, 'ECG Signal', 'ecg_fft.png')
    plot_fft(sample_resp, sample_fs_resp, 'Respiratory Signal', 'resp_fft.png')

    # Tạo báo cáo tóm tắt
    with open(os.path.join(figures_path, 'data_exploration_summary.txt'), 'w') as f:
        f.write("BÁO CÁO KHÁM PHÁ DỮ LIỆU BIDMC PPG AND RESPIRATION DATASET\n")
        f.write("==========================================================\n\n")

        f.write(f"Số lượng bản ghi: {len(data)}\n\n")

        f.write("Cấu trúc dữ liệu:\n")
        f.write("- Mỗi bản ghi chứa các trường: ppg, ekg, ref, fix\n")
        f.write("- Tín hiệu PPG và ECG được lưu trữ với giá trị (v) và tần số lấy mẫu (fs)\n")
        f.write("- Tín hiệu hô hấp được lưu trữ trong trường ref.resp_sig.imp\n")
        f.write("- Các thông số sinh lý (HR, RR, PR, SpO2) được lưu trữ trong trường ref.params\n\n")

        f.write(f"Tần số lấy mẫu PPG: {sample_fs_ppg} Hz\n")
        f.write(f"Tần số lấy mẫu ECG: {sample_fs_ecg} Hz\n")
        f.write(f"Tần số lấy mẫu Resp: {sample_fs_resp} Hz\n\n")

        f.write("Thách thức trong việc truy cập dữ liệu:\n")
        f.write("- Cấu trúc dữ liệu phức tạp với nhiều lớp lồng nhau\n")
        f.write("- Khó khăn trong việc chuyển đổi dữ liệu HR và RR sang định dạng float\n")
        f.write("- Cần phương pháp tiếp cận cẩn thận để trích xuất và xử lý dữ liệu\n\n")

        f.write("Các file đã tạo:\n")
        f.write("1. sample_signals.png - Biểu đồ mẫu của tín hiệu ECG, PPG và Respiratory\n")
        f.write("2. ppg_fft.png, ecg_fft.png, resp_fft.png - Phân tích phổ tần số của các tín hiệu\n")

    print("\nPhân tích dữ liệu hoàn tất. Các biểu đồ và báo cáo đã được lưu vào thư mục figures.")

except Exception as e:
    print(f"Lỗi khi vẽ tín hiệu mẫu: {e}")

# Kiểm tra trực tiếp một số bản ghi để tìm hiểu cấu trúc HR và RR
print("\nKiểm tra trực tiếp HR và RR từ một số bản ghi:")
for i in range(min(5, len(data))):
    try:
        record = data[i]
        params = record['ref'][0, 0]['params'][0, 0]

        hr_data = params['hr'][0]
        rr_data = params['rr'][0]

        print(f"\nBản ghi {i}:")
        print(f"Kiểu dữ liệu HR: {type(hr_data)}")
        if hasattr(hr_data, 'dtype'):
            print(f"Dtype của HR: {hr_data.dtype}")
        if hasattr(hr_data, 'shape'):
            print(f"Kích thước của HR: {hr_data.shape}")

        print(f"Kiểu dữ liệu RR: {type(rr_data)}")
        if hasattr(rr_data, 'dtype'):
            print(f"Dtype của RR: {rr_data.dtype}")
        if hasattr(rr_data, 'shape'):
            print(f"Kích thước của RR: {rr_data.shape}")

        # Thử in ra một số giá trị đầu tiên
        if hasattr(hr_data, 'size') and hr_data.size > 0:
            if hasattr(hr_data, 'dtype') and hr_data.dtype.names is not None and 'v' in hr_data.dtype.names:
                print(f"Giá trị HR đầu tiên (trường v): {hr_data['v'][0] if hr_data['v'].size > 0 else 'Không có dữ liệu'}")
            else:
                print(f"Giá trị HR đầu tiên: {hr_data[0]}")

        if hasattr(rr_data, 'size') and rr_data.size > 0:
            if hasattr(rr_data, 'dtype') and rr_data.dtype.names is not None and 'v' in rr_data.dtype.names:
                print(f"Giá trị RR đầu tiên (trường v): {rr_data['v'][0] if rr_data['v'].size > 0 else 'Không có dữ liệu'}")
            else:
                print(f"Giá trị RR đầu tiên: {rr_data[0]}")
    except Exception as e:
        print(f"Lỗi khi kiểm tra bản ghi {i}: {e}")


Streaming output truncated to the last 5000 lines.
       [21],
       [21],
       [21],
       [21],
       [21],
       [21],
       [21],
       [21],
       [21],
       [21],
       [20],
       [20],
       [20],
       [20],
       [20],
       [20],
       [20],
       [20],
       [20],
       [20],
       [20],
       [20],
       [20],
       [20],
       [20],
       [20],
       [20],
       [20],
       [20],
       [20],
       [20],
       [20],
       [21],
       [21],
       [21],
       [21],
       [21],
       [21],
       [21],
       [21],
       [21],
       [21],
       [21],
       [21],
       [21],
       [21],
       [21],
       [21],
       [21],
       [21],
       [21],
       [21],
       [21],
       [21],
       [21],
       [21],
       [21],
       [21],
       [21],
       [21],
       [21],
       [21],
       [21],
       [21],
       [21],
       [21],
       [21],
       [21],
       [21],
       [21],
       [21],
       [21],
       [21],


# **check_data_structure**

In [35]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.io as sio
import os
from scipy.fft import fft, fftfreq
import pandas as pd

# Đường dẫn đến file dữ liệu
data_path = '/content/drive/MyDrive/bidmc/bidmc-ppg-and-respiration-dataset-1.0.0/bidmc_data.mat'
figures_path = '/content/drive/MyDrive/bidmc/figures'

# Tải dữ liệu
print("Đang tải dữ liệu từ file .mat...")
mat_data = sio.loadmat(data_path)

# Kiểm tra cấu trúc dữ liệu
print("Các khóa trong file .mat:")
for key in mat_data.keys():
    print(f"- {key}")

# Kiểm tra cấu trúc chi tiết
if 'data' in mat_data:
    data = mat_data['data']
    print(f"\nKiểu dữ liệu của 'data': {type(data)}")
    print(f"Kích thước của 'data': {data.shape}")

    # Kiểm tra cấu trúc của phần tử đầu tiên nếu data là mảng
    if isinstance(data, np.ndarray) and data.size > 0:
        first_record = data[0]
        if hasattr(first_record, 'dtype') and hasattr(first_record.dtype, 'names'):
            print("\nCác trường trong bản ghi đầu tiên:")
            for field in first_record.dtype.names:
                print(f"- {field}")

                # Kiểm tra cấu trúc chi tiết của từng trường
                field_data = first_record[field]
                print(f"  Kiểu: {type(field_data)}, Kích thước: {field_data.shape}")

                # Nếu là mảng có cấu trúc, hiển thị thêm thông tin
                if hasattr(field_data, 'dtype') and hasattr(field_data.dtype, 'names'):
                    print(f"  Các trường con: {field_data.dtype.names}")
        else:
            print(f"\nPhần tử đầu tiên không có cấu trúc trường, kiểu: {type(first_record)}")
    else:
        print("\nKhông thể truy cập phần tử đầu tiên của 'data'")
else:
    print("\nKhông tìm thấy khóa 'data' trong file .mat")
    print("Đang kiểm tra cấu trúc dữ liệu thay thế...")

    # Tìm khóa có thể chứa dữ liệu chính
    potential_data_keys = [k for k in mat_data.keys() if not k.startswith('__')]
    for key in potential_data_keys:
        print(f"\nKiểm tra khóa: {key}")
        data_item = mat_data[key]
        print(f"Kiểu: {type(data_item)}, ", end="")

        if isinstance(data_item, np.ndarray):
            print(f"Kích thước: {data_item.shape}")

            # Kiểm tra nếu là mảng có cấu trúc
            if data_item.dtype.names is not None:
                print(f"Các trường: {data_item.dtype.names}")

            # Kiểm tra phần tử đầu tiên nếu là mảng nhiều chiều
            if data_item.size > 0:
                first_item = data_item[0]
                print(f"Kiểu phần tử đầu tiên: {type(first_item)}")

                # Nếu phần tử đầu tiên là mảng có cấu trúc
                if hasattr(first_item, 'dtype') and hasattr(first_item.dtype, 'names'):
                    print(f"Các trường trong phần tử đầu tiên: {first_item.dtype.names}")
        else:
            print(f"Không phải mảng numpy")

print("\nĐang lưu thông tin cấu trúc dữ liệu vào file...")
with open(os.path.join(figures_path, 'data_structure.txt'), 'w') as f:
    f.write("CẤU TRÚC DỮ LIỆU BIDMC PPG AND RESPIRATION DATASET\n")
    f.write("=================================================\n\n")

    f.write("Các khóa trong file .mat:\n")
    for key in mat_data.keys():
        f.write(f"- {key}\n")

    f.write("\nThông tin chi tiết về cấu trúc dữ liệu được lưu trong file này.")

print("\nĐã lưu thông tin cấu trúc dữ liệu. Vui lòng kiểm tra file data_structure.txt")


Đang tải dữ liệu từ file .mat...
Các khóa trong file .mat:
- __header__
- __version__
- __globals__
- data

Kiểu dữ liệu của 'data': <class 'numpy.ndarray'>
Kích thước của 'data': (1, 53)

Các trường trong bản ghi đầu tiên:
- fix
  Kiểu: <class 'numpy.ndarray'>, Kích thước: (53,)
  Các trường con: None
- ppg
  Kiểu: <class 'numpy.ndarray'>, Kích thước: (53,)
  Các trường con: None
- ekg
  Kiểu: <class 'numpy.ndarray'>, Kích thước: (53,)
  Các trường con: None
- ref
  Kiểu: <class 'numpy.ndarray'>, Kích thước: (53,)
  Các trường con: None

Đang lưu thông tin cấu trúc dữ liệu vào file...

Đã lưu thông tin cấu trúc dữ liệu. Vui lòng kiểm tra file data_structure.txt


# **preprocess_data_fixed**

In [32]:
import numpy as np
import scipy.io as sio
import os
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt

# Đường dẫn dữ liệu
data_path = '/content/drive/MyDrive/bidmc/bidmc-ppg-and-respiration-dataset-1.0.0/bidmc_data.mat'
processed_path = '/content/drive/MyDrive/bidmc/processed'
figures_path = '/content/drive/MyDrive/bidmc/figures'
os.makedirs(processed_path, exist_ok=True)
os.makedirs(figures_path, exist_ok=True)

# Tải dữ liệu
print("Đang tải dữ liệu từ file .mat...")
mat_data = sio.loadmat(data_path)
data = mat_data['data'][0]  # 53 bản ghi
print(f"Số lượng bản ghi: {len(data)}")

# Tham số
fs = 125  # Tần số lấy mẫu (Hz)
segment_length = 1250  # 10 giây (1250 mẫu)
overlap = 0.5  # 50% chồng lấp

# Hàm chuẩn hóa z-score từng cá nhân
def zscore_per_subject(data):
    mean = np.mean(data)
    std = np.std(data)
    if std == 0 or np.isnan(std):
        print(f"- Cảnh báo: Std = 0 hoặc nan, dữ liệu: {data[:5]}...")
        return np.zeros_like(data)  # Trả về mảng 0 nếu không chuẩn hóa được
    return (data - mean) / std

# Hàm kiểm tra và trích xuất mảng phẳng
def extract_flat_array(data, field='v', record_idx=None):
    try:
        if hasattr(data, 'dtype') and data.dtype.names is not None and field in data.dtype.names:
            values = data[field]
            print(f"- Bản ghi {record_idx}: Tìm thấy trường '{field}', shape ban đầu: {values.shape}")
        else:
            values = data
            print(f"- Bản ghi {record_idx}: Không có trường '{field}', shape ban đầu: {values.shape}")

        # Kiểm tra nội dung
        print(f"- Nội dung của '{field}' (5 phần tử đầu): {values[:5]}")

        # Xử lý mảng lồng nhau
        if values.size == 1 and isinstance(values[0], np.ndarray):
            flat_values = values[0].flatten()
            print(f"- Phát hiện mảng lồng nhau, shape sau khi trích xuất: {flat_values.shape}")
        else:
            flat_values = values.flatten()

        if len(flat_values) == 0:
            raise ValueError("Mảng trống sau khi flatten")

        # Kiểm tra giá trị nan
        if np.any(np.isnan(flat_values)):
            raise ValueError(f"Mảng chứa giá trị nan: {flat_values[:5]}...")

        return flat_values.astype(float)
    except Exception as e:
        raise ValueError(f"Lỗi trích xuất: {e}")

# Tiền xử lý dữ liệu
ppg_segments = []
hr_segments = []
rr_segments = []
valid_records = 0

for i, record in enumerate(data):
    try:
        # Trích xuất tín hiệu PPG
        ppg = record['ppg'][0, 0]['v'].flatten()
        print(f"\nXử lý bản ghi {i}:")
        print(f"- Độ dài tín hiệu PPG gốc: {len(ppg)} mẫu ({len(ppg)/fs:.2f} giây)")

        # Trích xuất HR và RR
        params = record['ref'][0, 0]['params'][0, 0]
        hr_values = extract_flat_array(params['hr'][0], 'v', i)
        rr_values = extract_flat_array(params['rr'][0], 'v', i)

        # Kiểm tra dữ liệu HR/RR
        if len(hr_values) < 2 or len(rr_values) < 2:
            print(f"- Không đủ dữ liệu HR/RR cho bản ghi {i} (HR: {len(hr_values)}, RR: {len(rr_values)}), bỏ qua")
            continue

        print(f"- Số giá trị HR gốc: {len(hr_values)}, Giá trị đầu tiên: {hr_values[0]:.2f}")
        print(f"- Số giá trị RR gốc: {len(rr_values)}, Giá trị đầu tiên: {rr_values[0]:.2f}")

        # Chuẩn hóa PPG về [0, 1]
        scaler = MinMaxScaler(feature_range=(0, 1))
        ppg_normalized = scaler.fit_transform(ppg.reshape(-1, 1)).flatten()

        # Chuẩn hóa HR/RR bằng z-score
        hr_normalized = zscore_per_subject(hr_values)
        rr_normalized = zscore_per_subject(rr_values)

        # Kiểm tra sau chuẩn hóa
        if np.any(np.isnan(hr_normalized)) or np.any(np.isnan(rr_normalized)):
            print(f"- Lỗi: HR hoặc RR chứa nan sau chuẩn hóa (HR: {hr_normalized[:5]}, RR: {rr_normalized[:5]})")
            continue

        # Phân đoạn tín hiệu
        step = int(segment_length * (1 - overlap))
        num_segments = (len(ppg) - segment_length) // step + 1
        print(f"- Số đoạn tín hiệu: {num_segments}")

        hr_fs = len(hr_values) / (len(ppg) / fs)  # Tần số lấy mẫu của HR/RR
        samples_per_segment = int(segment_length / fs * hr_fs)

        for j in range(num_segments):
            start = j * step
            end = start + segment_length

            if end <= len(ppg):
                ppg_seg = ppg_normalized[start:end]

                hr_start_idx = int(start / fs * hr_fs)
                hr_end_idx = min(int(end / fs * hr_fs), len(hr_values))
                rr_start_idx = int(start / fs * hr_fs)
                rr_end_idx = min(int(end / fs * hr_fs), len(rr_values))

                hr_seg = np.mean(hr_normalized[hr_start_idx:hr_end_idx]) if hr_end_idx > hr_start_idx else hr_normalized[hr_start_idx]
                rr_seg = np.mean(rr_normalized[rr_start_idx:rr_end_idx]) if rr_end_idx > rr_start_idx else rr_normalized[rr_start_idx]

                if np.isnan(hr_seg) or np.isnan(rr_seg):
                    print(f"- Lỗi: Đoạn {j} chứa nan (HR: {hr_seg}, RR: {rr_seg})")
                    continue

                ppg_segments.append(ppg_seg)
                hr_segments.append(hr_seg)
                rr_segments.append(rr_seg)

        valid_records += 1
        print(f"- Đã xử lý thành công bản ghi {i}, thêm {num_segments} đoạn")

    except Exception as e:
        print(f"- Lỗi khi xử lý bản ghi {i}: {e}")

# Chuyển thành mảng numpy
ppg_segments = np.array(ppg_segments)
hr_segments = np.array(hr_segments)
rr_segments = np.array(rr_segments)

# Kiểm tra dữ liệu
print(f"\nTổng kết tiền xử lý:")
print(f"- Số bản ghi xử lý thành công: {valid_records}/{len(data)}")
print(f"- Tổng số đoạn tín hiệu: {len(ppg_segments)}")
if len(ppg_segments) == 0:
    print("Không có dữ liệu hợp lệ. Tạo dữ liệu giả lập để minh họa.")
    num_samples = 100
    ppg_segments = np.random.rand(num_samples, segment_length)
    hr_segments = zscore_per_subject(np.random.uniform(60, 120, num_samples))
    rr_segments = zscore_per_subject(np.random.uniform(6, 18, num_samples))
    print(f"- Đã tạo {num_samples} mẫu dữ liệu giả lập.")

# Chia dữ liệu thành train/test (80/20)
train_size = int(0.8 * len(ppg_segments))
indices = np.random.permutation(len(ppg_segments))
train_indices = indices[:train_size]
test_indices = indices[train_size:]

X_train = ppg_segments[train_indices]
X_test = ppg_segments[test_indices]
hr_train = hr_segments[train_indices]
hr_test = hr_segments[test_indices]
rr_train = rr_segments[train_indices]
rr_test = rr_segments[test_indices]

# In kích thước dữ liệu
print(f"- Kích thước tập huấn luyện: {X_train.shape}")
print(f"- Kích thước tập kiểm thử: {X_test.shape}")

# Lưu dữ liệu
np.save(os.path.join(processed_path, 'ppg_train.npy'), X_train)
np.save(os.path.join(processed_path, 'ppg_test.npy'), X_test)
np.save(os.path.join(processed_path, 'hr_train.npy'), hr_train)
np.save(os.path.join(processed_path, 'hr_test.npy'), hr_test)
np.save(os.path.join(processed_path, 'rr_train.npy'), rr_train)
np.save(os.path.join(processed_path, 'rr_test.npy'), rr_test)

# Ghi thông tin tiền xử lý vào file
with open(os.path.join(processed_path, 'preprocessing_info.txt'), 'w') as f:
    f.write("THÔNG TIN TIỀN XỬ LÝ DỮ LIỆU\n")
    f.write("============================\n\n")
    f.write(f"Số bản ghi xử lý thành công: {valid_records}/{len(data)}\n")
    f.write(f"Tổng số đoạn tín hiệu: {len(ppg_segments)}\n\n")
    f.write("Tham số tiền xử lý:\n")
    f.write(f"- Tần số lấy mẫu: {fs} Hz\n")
    f.write(f"- Độ dài đoạn tín hiệu: {segment_length} mẫu ({segment_length/fs} giây)\n")
    f.write(f"- Độ chồng lấp: {overlap*100}%\n\n")
    f.write("Kích thước dữ liệu:\n")
    f.write(f"- Tập huấn luyện: {X_train.shape[0]} mẫu\n")
    f.write(f"- Tập kiểm thử: {X_test.shape[0]} mẫu\n\n")
    f.write("Thống kê HR (z-score normalized):\n")
    f.write(f"- Min: {np.min(hr_segments):.4f}, Max: {np.max(hr_segments):.4f}\n")
    f.write(f"- Mean: {np.mean(hr_segments):.4f}, Std: {np.std(hr_segments):.4f}\n\n")
    f.write("Thống kê RR (z-score normalized):\n")
    f.write(f"- Min: {np.min(rr_segments):.4f}, Max: {np.max(rr_segments):.4f}\n")
    f.write(f"- Mean: {np.mean(rr_segments):.4f}, Std: {np.std(rr_segments):.4f}\n")
    if valid_records == 0:
        f.write("\nLưu ý: Dữ liệu là giả lập do không trích xuất được từ BIDMC.\n")

# Vẽ phân phối HR và RR
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.hist(hr_segments, bins=30, alpha=0.7, color='blue')
plt.axvline(np.mean(hr_segments), color='red', linestyle='dashed', linewidth=1)
plt.title('Heart Rate Distribution (z-score)')
plt.xlabel('HR (normalized)')
plt.ylabel('Frequency')
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
plt.hist(rr_segments, bins=30, alpha=0.7, color='green')
plt.axvline(np.mean(rr_segments), color='red', linestyle='dashed', linewidth=1)
plt.title('Respiratory Rate Distribution (z-score)')
plt.xlabel('RR (normalized)')
plt.ylabel('Frequency')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(os.path.join(figures_path, 'hr_rr_distribution.png'))
plt.close()

# Vẽ một số đoạn PPG
plt.figure(figsize=(15, 10))
for i in range(min(5, len(X_train))):
    plt.subplot(5, 1, i+1)
    plt.plot(X_train[i])
    plt.title(f'Preprocessed PPG Segment {i+1}')
    plt.xlabel('Sample')
    plt.ylabel('Amplitude (Normalized [0, 1])')
    plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(os.path.join(figures_path, 'preprocessed_ppg_segments.png'))
plt.close()

print("\nTiền xử lý dữ liệu hoàn tất. Dữ liệu đã được lưu vào thư mục processed.")

Streaming output truncated to the last 5000 lines.
        [19],
        [19],
        [19],
        [19],
        [19],
        [19],
        [19],
        [19],
        [19],
        [19],
        [19],
        [19],
        [19],
        [19],
        [19],
        [19],
        [19],
        [19],
        [19],
        [19],
        [19],
        [19],
        [19],
        [19],
        [19],
        [20],
        [20],
        [20],
        [20],
        [20],
        [20],
        [20],
        [20],
        [20],
        [20],
        [20],
        [20],
        [20],
        [20],
        [20],
        [20],
        [20],
        [20],
        [20],
        [20],
        [20],
        [20],
        [20],
        [20],
        [20],
        [20],
        [20],
        [20],
        [20],
        [20],
        [20],
        [20],
        [20],
        [20],
        [20],
        [20],
        [20],
        [20],
        [20],
        [20],
        [20],
        [20],
        [20

# **cvae_model**

In [36]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os
from tensorflow.keras import layers, Model
from tensorflow.keras.optimizers import Adam
import datetime

# Đường dẫn đến dữ liệu đã tiền xử lý
processed_data_path = '/content/drive/MyDrive/bidmc/processed'
model_path = '/content/drive/MyDrive/bidmc/models'
figures_path = '/content/drive/MyDrive/bidmc/figures'

# Tạo thư mục nếu chưa tồn tại
os.makedirs(model_path, exist_ok=True)
os.makedirs(figures_path, exist_ok=True)

# Tải dữ liệu đã tiền xử lý
print("Đang tải dữ liệu đã tiền xử lý...")
X_train = np.load(os.path.join(processed_data_path, 'ppg_train.npy'))
X_test = np.load(os.path.join(processed_data_path, 'ppg_test.npy'))
hr_train = np.load(os.path.join(processed_data_path, 'hr_train.npy'))
hr_test = np.load(os.path.join(processed_data_path, 'hr_test.npy'))
rr_train = np.load(os.path.join(processed_data_path, 'rr_train.npy'))
rr_test = np.load(os.path.join(processed_data_path, 'rr_test.npy'))

print(f"Kích thước dữ liệu huấn luyện: {X_train.shape}")
print(f"Kích thước dữ liệu kiểm thử: {X_test.shape}")

# Tham số mô hình
input_dim = X_train.shape[1]  # Độ dài đoạn tín hiệu PPG
condition_dim = 2  # HR và RR
latent_dim = 64  # Kích thước không gian tiềm ẩn
hidden_units = [256, 128, 64]  # Số đơn vị ẩn trong các lớp
batch_size = 64
epochs = 100
learning_rate = 0.0001

# Định nghĩa lớp Sampling để lấy mẫu từ không gian tiềm ẩn
class Sampling(layers.Layer):
    def call(self, inputs):
        z_mean, z_log_var = inputs
        batch = tf.shape(z_mean)[0]
        dim = tf.shape(z_mean)[1]
        epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
        return z_mean + tf.exp(0.5 * z_log_var) * epsilon

# Xây dựng Encoder
def build_encoder(input_dim, condition_dim, latent_dim, hidden_units):
    # Đầu vào tín hiệu PPG
    encoder_inputs = layers.Input(shape=(input_dim,), name='encoder_input')

    # Đầu vào điều kiện (HR và RR)
    condition_inputs = layers.Input(shape=(condition_dim,), name='condition_input')

    # Kết hợp đầu vào tín hiệu và điều kiện
    x = layers.Concatenate()([encoder_inputs, condition_inputs])

    # Các lớp ẩn
    for units in hidden_units:
        x = layers.Dense(units, activation='relu')(x)

    # Lớp đầu ra
    z_mean = layers.Dense(latent_dim, name='z_mean')(x)
    z_log_var = layers.Dense(latent_dim, name='z_log_var')(x)

    # Lấy mẫu từ không gian tiềm ẩn
    z = Sampling()([z_mean, z_log_var])

    # Định nghĩa mô hình
    encoder = Model([encoder_inputs, condition_inputs], [z_mean, z_log_var, z], name='encoder')

    return encoder

# Xây dựng Decoder
def build_decoder(latent_dim, condition_dim, input_dim, hidden_units):
    # Đầu vào từ không gian tiềm ẩn
    latent_inputs = layers.Input(shape=(latent_dim,), name='latent_input')

    # Đầu vào điều kiện (HR và RR)
    condition_inputs = layers.Input(shape=(condition_dim,), name='condition_input')

    # Kết hợp đầu vào từ không gian tiềm ẩn và điều kiện
    x = layers.Concatenate()([latent_inputs, condition_inputs])

    # Các lớp ẩn
    for units in reversed(hidden_units):
        x = layers.Dense(units, activation='relu')(x)

    # Lớp đầu ra
    decoder_outputs = layers.Dense(input_dim, activation='tanh')(x)

    # Định nghĩa mô hình
    decoder = Model([latent_inputs, condition_inputs], decoder_outputs, name='decoder')

    return decoder

# Xây dựng mô hình CVAE
class CVAE(Model):
    def __init__(self, encoder, decoder, **kwargs):
        super(CVAE, self).__init__(**kwargs)
        self.encoder = encoder
        self.decoder = decoder
        self.total_loss_tracker = tf.keras.metrics.Mean(name="total_loss")
        self.reconstruction_loss_tracker = tf.keras.metrics.Mean(name="reconstruction_loss")
        self.kl_loss_tracker = tf.keras.metrics.Mean(name="kl_loss")

    @property
    def metrics(self):
        return [
            self.total_loss_tracker,
            self.reconstruction_loss_tracker,
            self.kl_loss_tracker,
        ]

    def train_step(self, data):
        x, condition = data
        print("x shape:", x.shape)
        print("condition shape:", condition.shape)

        with tf.GradientTape() as tape:
            z_mean, z_log_var, z = self.encode([x, condition])
            reconstruction = self.decode([z, condition])
            print("reconstruction shape:", reconstruction.shape)

            mse_result = tf.keras.losses.mse(x, reconstruction)
            print("mse result shape:", mse_result.shape)

            reconstruction_loss = tf.reduce_mean(
                tf.reduce_sum(mse_result, axis=1)
            )
            kl_loss = -0.5 * tf.reduce_mean(
                tf.reduce_sum(1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var), axis=1)
            )
            total_loss = reconstruction_loss + kl_loss

        grads = tape.gradient(total_loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(grads, self.trainable_weights))

        self.total_loss_tracker.update_state(total_loss)
        self.reconstruction_loss_tracker.update_state(reconstruction_loss)
        self.kl_loss_tracker.update_state(kl_loss)

        return {
            "loss": self.total_loss_tracker.result(),
            "reconstruction_loss": self.reconstruction_loss_tracker.result(),
            "kl_loss": self.kl_loss_tracker.result(),
        }

    def test_step(self, data):
        if isinstance(data, tuple):
            x_condition = data[0]
            if isinstance(x_condition, list) and len(x_condition) == 2:
                x, condition = x_condition
            else:
                raise ValueError("Input data format is incorrect. Expected a list with [x, condition]")
        else:
            raise ValueError("Input data format is incorrect. Expected a tuple with ([x, condition], None)")

        # Encoder
        z_mean, z_log_var, z = self.encoder([x, condition])

        # Decoder
        reconstruction = self.decoder([z, condition])

        # Tính toán loss
        reconstruction_loss = tf.reduce_mean(
            tf.reduce_sum(
                tf.keras.losses.mse(x, reconstruction), axis=1
            )
        )

        kl_loss = -0.5 * tf.reduce_mean(
            tf.reduce_sum(
                1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var), axis=1
            )
        )

        total_loss = reconstruction_loss + kl_loss

        # Cập nhật metrics
        self.total_loss_tracker.update_state(total_loss)
        self.reconstruction_loss_tracker.update_state(reconstruction_loss)
        self.kl_loss_tracker.update_state(kl_loss)

        return {
            "loss": self.total_loss_tracker.result(),
            "reconstruction_loss": self.reconstruction_loss_tracker.result(),
            "kl_loss": self.kl_loss_tracker.result(),
        }

    def call(self, inputs):
        x, condition = inputs
        z_mean, z_log_var, z = self.encoder([x, condition])
        reconstruction = self.decoder([z, condition])
        return reconstruction

    def generate(self, condition, z=None):
        if z is None:
            # Tạo vector ngẫu nhiên từ không gian tiềm ẩn
            z = tf.random.normal(shape=(condition.shape[0], latent_dim))

        # Tạo tín hiệu PPG từ vector z và điều kiện
        return self.decoder([z, condition])

# Xây dựng mô hình
print("Đang xây dựng mô hình CVAE...")
encoder = build_encoder(input_dim, condition_dim, latent_dim, hidden_units)
decoder = build_decoder(latent_dim, condition_dim, input_dim, hidden_units)
cvae = CVAE(encoder, decoder)

# Biên dịch mô hình
cvae.compile(optimizer=Adam(learning_rate=learning_rate))

# Tóm tắt mô hình
print("Tóm tắt mô hình Encoder:")
encoder.summary()
print("\nTóm tắt mô hình Decoder:")
decoder.summary()

# Chuẩn bị dữ liệu điều kiện
condition_train = np.column_stack((hr_train, rr_train))
condition_test = np.column_stack((hr_test, rr_test))

# Tạo TensorBoard callback
log_dir = os.path.join(model_path, "logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

# Tạo ModelCheckpoint callback
checkpoint_path = os.path.join(model_path, "cvae_checkpoint.weights.h5")
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path,
    save_weights_only=True,
    monitor='loss',
    mode='min',
    save_best_only=True
)

# Tạo EarlyStopping callback
early_stopping_callback = tf.keras.callbacks.EarlyStopping(
    monitor='loss',
    patience=5,
    restore_best_weights=True
)

# Lưu thông tin mô hình
with open(os.path.join(model_path, 'model_info.txt'), 'w') as f:
    f.write("THÔNG TIN MÔ HÌNH CVAE\n")
    f.write("=====================\n\n")

    f.write("Tham số mô hình:\n")
    f.write(f"- Kích thước đầu vào: {input_dim}\n")
    f.write(f"- Kích thước điều kiện: {condition_dim}\n")
    f.write(f"- Kích thước không gian tiềm ẩn: {latent_dim}\n")
    f.write(f"- Số đơn vị ẩn trong các lớp: {hidden_units}\n")
    f.write(f"- Kích thước batch: {batch_size}\n")
    f.write(f"- Số epoch: {epochs}\n")
    f.write(f"- Tốc độ học: {learning_rate}\n\n")

    f.write("Kích thước dữ liệu:\n")
    f.write(f"- Tập huấn luyện: {X_train.shape[0]} mẫu\n")
    f.write(f"- Tập kiểm thử: {X_test.shape[0]} mẫu\n")

print("\nMô hình CVAE đã được xây dựng thành công.")
print("Thông tin mô hình đã được lưu vào file model_info.txt.")
print("Sẵn sàng để huấn luyện mô hình.")


Đang tải dữ liệu đã tiền xử lý...
Kích thước dữ liệu huấn luyện: (3724, 1250)
Kích thước dữ liệu kiểm thử: (931, 1250)
Đang xây dựng mô hình CVAE...
Tóm tắt mô hình Encoder:


Model: "encoder"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ encoder_input             │ (None, 1250)           │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ condition_input           │ (None, 2)              │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ concatenate_45            │ (None, 1252)           │              0 │ encoder_input[0][0],   │
│ (Concatenate)             │                        │                │ condition_input[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_104 (Dense)         │ (None, 256)            │        320,768 │ concatenate_45[0][0]   │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_105 (Dense)         │ (None, 128)            │         32,896 │ dense_104[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_106 (Dense)         │ (None, 64)             │          8,256 │ dense_105[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ z_mean (Dense)            │ (None, 64)             │          4,160 │ dense_106[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ z_log_var (Dense)         │ (None, 64)             │          4,160 │ dense_106[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ sampling_22 (Sampling)    │ (None, 64)             │              0 │ z_mean[0][0],          │
│                           │                        │                │ z_log_var[0][0]        │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 370,240 (1.41 MB)

 Trainable params: 370,240 (1.41 MB)

...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ latent_input (InputLayer) │ (None, 64)             │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ condition_input           │ (None, 2)              │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ concatenate_46            │ (None, 66)             │              0 │ latent_input[0][0],    │
│ (Concatenate)             │                        │                │ condition_input[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_107 (Dense)         │ (None, 64)             │          4,288 │ concatenate_46[0][0]   │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_108 (Dense)         │ (None, 128)            │          8,320 │ dense_107[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_109 (Dense)         │ (None, 256)            │         33,024 │ dense_108[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_110 (Dense)         │ (None, 1250)           │        321,250 │ dense_109[0][0]        │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 366,882 (1.40 MB)

 Trainable params: 366,882 (1.40 MB)

 Non-trainable params: 0 (0.00 B)


Mô hình CVAE đã được xây dựng thành công.
Thông tin mô hình đã được lưu vào file model_info.txt.
Sẵn sàng để huấn luyện mô hình.


# **Dense_CVAE_v3.1.py**

In [37]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os
import datetime
from tensorflow.keras import layers, Model
from tensorflow.keras.optimizers import Adam
from scipy.signal import find_peaks
from scipy.fft import fft
import time

# Đường dẫn đến dữ liệu và lưu trữ
processed_data_path = '/content/drive/MyDrive/bidmc/processed'
model_path = '/content/drive/MyDrive/bidmc/models'
figures_path = '/content/drive/MyDrive/bidmc/figures'

# Tạo thư mục nếu chưa tồn tại
os.makedirs(processed_data_path, exist_ok=True)
os.makedirs(model_path, exist_ok=True)
os.makedirs(figures_path, exist_ok=True)

# Tải dữ liệu đã tiền xử lý từ bước trước
print("Đang tải dữ liệu đã tiền xử lý từ bước trước...")
X_train = np.load(os.path.join(processed_data_path, 'ppg_train.npy'))
X_test = np.load(os.path.join(processed_data_path, 'ppg_test.npy'))
hr_train = np.load(os.path.join(processed_data_path, 'hr_train.npy'))
hr_test = np.load(os.path.join(processed_data_path, 'hr_test.npy'))
rr_train = np.load(os.path.join(processed_data_path, 'rr_train.npy'))
rr_test = np.load(os.path.join(processed_data_path, 'rr_test.npy'))

print(f"Kích thước dữ liệu huấn luyện: {X_train.shape}")
print(f"Kích thước dữ liệu kiểm thử: {X_test.shape}")

# Chuẩn hóa dữ liệu
X_train = (X_train - np.min(X_train)) / (np.max(X_train) - np.min(X_train))
X_test = (X_test - np.min(X_test)) / (np.max(X_test) - np.min(X_test))
condition_train = tf.convert_to_tensor(np.column_stack((hr_train, rr_train)), dtype=tf.float32)
condition_test = tf.convert_to_tensor(np.column_stack((hr_test, rr_test)), dtype=tf.float32)
condition_train = (condition_train - tf.reduce_min(condition_train, axis=0)) / (tf.reduce_max(condition_train, axis=0) - tf.reduce_min(condition_train, axis=0))
condition_test = (condition_test - tf.reduce_min(condition_test, axis=0)) / (tf.reduce_max(condition_test, axis=0) - tf.reduce_min(condition_test, axis=0))

# Tham số mô hình
input_dim = X_train.shape[1]  # 1250
condition_dim = 2  # HR và RR
latent_dim = 20
hidden_units = [256, 128]
batch_size = 64  # Tăng batch_size để ổn định hơn
epochs = 100
learning_rate = 0.0001  # Giảm learning_rate

# Định nghĩa lớp Sampling
class Sampling(layers.Layer):
    def call(self, inputs):
        z_mean, z_log_var = inputs
        batch = tf.shape(z_mean)[0]
        dim = tf.shape(z_mean)[1]
        epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
        return z_mean + tf.exp(0.5 * z_log_var) * epsilon

# Xây dựng mô hình CVAE (chỉ Dense)
class CVAE(tf.keras.Model):
    def __init__(self, input_dim, condition_dim, latent_dim, hidden_units):
        super(CVAE, self).__init__()
        self.input_dim = input_dim
        self.condition_dim = condition_dim
        self.latent_dim = latent_dim
        self.hidden_units = hidden_units

        # Xây dựng Encoder
        encoder_inputs = layers.Input(shape=(input_dim,), name='encoder_input')
        condition_inputs = layers.Input(shape=(condition_dim,), name='condition_input')
        x = layers.Concatenate()([encoder_inputs, condition_inputs])
        for units in hidden_units:
            x = layers.Dense(units)(x)
            x = layers.BatchNormalization()(x)
            x = layers.ReLU()(x)
        z_mean = layers.Dense(latent_dim, name='z_mean')(x)
        z_log_var = layers.Dense(latent_dim, name='z_log_var')(x)
        z = Sampling()([z_mean, z_log_var])
        self.encoder = Model([encoder_inputs, condition_inputs], [z_mean, z_log_var, z], name='encoder')

        # Xây dựng Decoder
        latent_inputs = layers.Input(shape=(latent_dim,), name='latent_input')
        condition_inputs_decoder = layers.Input(shape=(condition_dim,), name='condition_input')
        x = layers.Concatenate()([latent_inputs, condition_inputs_decoder])
        for units in reversed(hidden_units):
            x = layers.Dense(units)(x)
            x = layers.BatchNormalization()(x)
            x = layers.ReLU()(x)
        decoder_outputs = layers.Dense(input_dim, activation='sigmoid')(x)
        self.decoder = Model([latent_inputs, condition_inputs_decoder], decoder_outputs, name='decoder')

        # Metrics
        self.total_loss_tracker = tf.keras.metrics.Mean(name="total_loss")
        self.reconstruction_loss_tracker = tf.keras.metrics.Mean(name="reconstruction_loss")
        self.kl_loss_tracker = tf.keras.metrics.Mean(name="kl_loss")

    @property
    def metrics(self):
        return [self.total_loss_tracker, self.reconstruction_loss_tracker, self.kl_loss_tracker]

    def encode(self, inputs):
        return self.encoder(inputs)

    def decode(self, inputs):
        return self.decoder(inputs)

    def call(self, inputs):
        x, condition = inputs
        z_mean, z_log_var, z = self.encode([x, condition])
        return self.decode([z, condition])

    def train_step(self, data):
        x, condition = data
        with tf.GradientTape() as tape:
            z_mean, z_log_var, z = self.encode([x, condition])
            # Clip z_log_var trong train_step để tránh bùng nổ
            z_log_var = tf.clip_by_value(z_log_var, -10.0, 10.0)
            reconstruction = self.decode([z, condition])
            reconstruction_loss = tf.reduce_mean(tf.keras.losses.mse(x, reconstruction))
            kl_loss = -0.5 * tf.reduce_mean(tf.reduce_sum(1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var), axis=1))
            total_loss = reconstruction_loss + kl_loss

            # Kiểm tra nan
            def handle_nan():
                return {
                    "loss": tf.constant(float('nan')),
                    "reconstruction_loss": tf.constant(float('nan')),
                    "kl_loss": tf.constant(float('nan'))
                }

            def handle_valid():
                grads = tape.gradient(total_loss, self.trainable_weights)
                # Clip gradient để tránh bùng nổ
                grads = [tf.clip_by_norm(g, 1.0) if g is not None else g for g in grads]
                self.optimizer.apply_gradients(zip(grads, self.trainable_weights))
                self.total_loss_tracker.update_state(total_loss)
                self.reconstruction_loss_tracker.update_state(reconstruction_loss)
                self.kl_loss_tracker.update_state(kl_loss)
                return {
                    "loss": self.total_loss_tracker.result(),
                    "reconstruction_loss": self.reconstruction_loss_tracker.result(),
                    "kl_loss": self.kl_loss_tracker.result()
                }

            result = tf.cond(
                tf.math.is_nan(total_loss),
                handle_nan,
                handle_valid
            )
        return result

    def test_step(self, data):
        x, condition = data
        z_mean, z_log_var, z = self.encode([x, condition])
        # Clip z_log_var trong test_step
        z_log_var = tf.clip_by_value(z_log_var, -10.0, 10.0)
        reconstruction = self.decode([z, condition])
        reconstruction_loss = tf.reduce_mean(tf.keras.losses.mse(x, reconstruction))
        kl_loss = -0.5 * tf.reduce_mean(tf.reduce_sum(1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var), axis=1))
        total_loss = reconstruction_loss + kl_loss
        self.total_loss_tracker.update_state(total_loss)
        self.reconstruction_loss_tracker.update_state(reconstruction_loss)
        self.kl_loss_tracker.update_state(kl_loss)
        return {
            "loss": self.total_loss_tracker.result(),
            "reconstruction_loss": self.reconstruction_loss_tracker.result(),
            "kl_loss": self.kl_loss_tracker.result()
        }

    def generate(self, condition, z=None):
        condition = tf.convert_to_tensor(condition, dtype=tf.float32)
        if z is None:
            batch_size = tf.shape(condition)[0]
            z = tf.random.normal(shape=(batch_size, self.latent_dim))
        return self.decode([z, condition])

# Đánh giá định lượng và định tính
def evaluate_and_plot(cvae, X_test, condition_test, figures_path, model_type='dense', fs=125):
    ppg_generated = cvae.generate(condition_test).numpy()
    mse = np.mean(np.square(X_test - ppg_generated))
    hr_extracted = [len(find_peaks(ppg, distance=fs*0.6)[0]) * (60 / 10) for ppg in ppg_generated]
    hr_mae = np.mean(np.abs(hr_extracted - hr_test))
    br_extracted = []
    for ppg in ppg_generated:
        fft_vals = np.abs(fft(ppg))
        freqs = np.fft.fftfreq(len(ppg), 1/fs)
        br_idx = (freqs > 0.1) & (freqs < 0.5)
        br = freqs[np.argmax(fft_vals[br_idx])] * 60 if np.any(br_idx) else 0
        br_extracted.append(br)
    br_mae = np.mean(np.abs(br_extracted - rr_test))
    print(f"DENSE - MSE: {mse:.4f}, HR MAE: {hr_mae:.2f} bpm, BR MAE: {br_mae:.2f} breaths/min")
    plt.figure(figsize=(15, 10))
    for i in range(3):
        plt.subplot(3, 2, 2*i+1)
        plt.plot(X_test[i], label=f'Real PPG (HR={hr_test[i]:.1f}, BR={rr_test[i]:.1f})')
        plt.legend()
        plt.subplot(3, 2, 2*i+2)
        plt.plot(ppg_generated[i], label=f'Gen PPG (HR={hr_extracted[i]:.1f}, BR={br_extracted[i]:.1f})', color='orange')
        plt.legend()
    plt.tight_layout()
    plt.savefig(os.path.join(figures_path, 'dense_ppg_comparison_v3.png'))
    plt.close()
    return mse, hr_mae, br_mae

# Xây dựng và huấn luyện mô hình Dense
print("\nĐang xây dựng mô hình CVAE (DENSE)...")
cvae = CVAE(input_dim, condition_dim, latent_dim, hidden_units)
cvae.build(input_shape=[(None, input_dim), (None, condition_dim)])
cvae.compile(optimizer=Adam(learning_rate=learning_rate))

# Tạo dataset với shuffle
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, condition_train)).shuffle(buffer_size=1024).batch(batch_size)
test_dataset = tf.data.Dataset.from_tensor_slices((X_test, condition_test)).batch(batch_size)
X_input = X_train
X_test_input = X_test

# Callbacks
log_dir = os.path.join(model_path, "logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
checkpoint_path = os.path.join(model_path, "cvae_dense_checkpoint_v3.weights.h5")
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path,
    save_weights_only=True,
    monitor='loss',
    mode='min',
    save_best_only=True
)
early_stopping_callback = tf.keras.callbacks.EarlyStopping(
    monitor='loss',
    patience=10,
    restore_best_weights=True
)

# Huấn luyện mô hình
print("\nBắt đầu huấn luyện mô hình DENSE...")
start_time = time.time()
history = cvae.fit(
    train_dataset,
    epochs=epochs,
    validation_data=test_dataset,
    callbacks=[tensorboard_callback, checkpoint_callback, early_stopping_callback],
    verbose=1
)
training_time = time.time() - start_time
print(f"\nHuấn luyện DENSE hoàn tất trong {training_time:.2f} giây.")

# Lưu mô hình
cvae.save_weights(os.path.join(model_path, 'cvae_dense_final_v3.weights.h5'))
print(f"Đã lưu mô hình DENSE tại: {os.path.join(model_path, 'cvae_dense_final_v3.weights.h5')}")

# Đánh giá và vẽ biểu đồ
mse, hr_mae, br_mae = evaluate_and_plot(cvae, X_test_input, condition_test, figures_path)

# Vẽ biểu đồ loss
plt.figure(figsize=(15, 5))
plt.subplot(1, 3, 1)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('DENSE Total Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(['Train', 'Validation'], loc='upper right')
plt.grid(True, alpha=0.3)
plt.subplot(1, 3, 2)
plt.plot(history.history['reconstruction_loss'])
plt.title('DENSE Reconstruction Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.grid(True, alpha=0.3)
plt.subplot(1, 3, 3)
plt.plot(history.history['kl_loss'])
plt.title('DENSE KL Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig(os.path.join(figures_path, 'dense_training_history_v3.png'))
plt.close()

# Lưu thông tin huấn luyện
with open(os.path.join(model_path, 'training_info_dense_v3.txt'), 'w') as f:
    f.write("THÔNG TIN HUẤN LUYỆN MÔ HÌNH CVAE (DENSE V3.6)\n")
    f.write("=================================\n\n")
    f.write("Tham số huấn luyện:\n")
    f.write(f"- Kích thước batch: {batch_size}\n")
    f.write(f"- Số epoch: {epochs}\n")
    f.write(f"- Tốc độ học: {learning_rate}\n\n")
    f.write("Kết quả huấn luyện:\n")
    f.write(f"- Số epoch đã huấn luyện: {len(history.history['loss'])}\n")
    f.write(f"- Loss cuối cùng (train): {history.history['loss'][-1]:.4f}\n")
    f.write(f"- Loss cuối cùng (validation): {history.history['val_loss'][-1]:.4f}\n")
    f.write(f"- Reconstruction loss cuối cùng: {history.history['reconstruction_loss'][-1]:.4f}\n")
    f.write(f"- KL loss cuối cùng: {history.history['kl_loss'][-1]:.4f}\n")
    f.write(f"- Thời gian huấn luyện: {training_time:.2f} giây\n")
    f.write(f"- MSE (test): {mse:.4f}\n")
    f.write(f"- HR MAE (test): {hr_mae:.2f} bpm\n")
    f.write(f"- BR MAE (test): {br_mae:.2f} breaths/min\n\n")
    f.write("Đường dẫn đến mô hình đã lưu:\n")
    f.write(f"- Mô hình cuối cùng: {os.path.join(model_path, 'cvae_dense_final_v3.weights.h5')}\n")
    f.write(f"- Mô hình checkpoint: {checkpoint_path}\n")

print("\nQuá trình huấn luyện DENSE đã hoàn tất. Thông tin huấn luyện đã được lưu vào file training_info_dense_v3.txt.")

Đang tải dữ liệu đã tiền xử lý từ bước trước...
Kích thước dữ liệu huấn luyện: (3724, 1250)
Kích thước dữ liệu kiểm thử: (931, 1250)

Đang xây dựng mô hình CVAE (DENSE)...

Bắt đầu huấn luyện mô hình DENSE...
Epoch 1/100
59/59 ━━━━━━━━━━━━━━━━━━━━ 27s 156ms/step - kl_loss: 0.5593 - loss: 0.5926 - reconstruction_loss: 0.0333 - val_kl_loss: 0.0088 - val_loss: 0.0409 - val_reconstruction_loss: 0.0321
Epoch 2/100
59/59 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - kl_loss: 0.0048 - loss: 0.0364 - reconstruction_loss: 0.0316 - val_kl_loss: 8.9032e-04 - val_loss: 0.0323 - val_reconstruction_loss: 0.0314
Epoch 3/100
59/59 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - kl_loss: 5.7826e-04 - loss: 0.0317 - reconstruction_loss: 0.0311 - val_kl_loss: 3.3201e-04 - val_loss: 0.0316 - val_reconstruction_loss: 0.0312
Epoch 4/100
59/59 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - kl_loss: 2.4848e-04 - loss: 0.0313 - reconstruction_loss: 0.0311 - val_kl_loss: 1.7023e-04 - val_loss: 0.0314 - val_reconstruction_loss: 0.0312
Epoch 5/1

# **analyze_fourier**

In [39]:
import numpy as np
import matplotlib.pyplot as plt
import os
from scipy.signal import welch, find_peaks
from scipy.fft import fft, fftfreq
import pandas as pd
from sklearn.metrics import mean_squared_error
import tensorflow as tf
from tensorflow.keras import layers, Model
from tensorflow.keras.optimizers import Adam

# Đường dẫn đến dữ liệu và kết quả
processed_data_path = '/content/drive/MyDrive/bidmc/processed'
model_path = '/content/drive/MyDrive/bidmc/models'
figures_path = '/content/drive/MyDrive/bidmc/figures/fourier'
results_path = '/content/drive/MyDrive/bidmc/results'

# Tạo thư mục nếu chưa tồn tại
os.makedirs(figures_path, exist_ok=True)
os.makedirs(results_path, exist_ok=True)

# Tải dữ liệu kiểm thử
print("Đang tải dữ liệu kiểm thử...")
X_test = np.load(os.path.join(processed_data_path, 'ppg_test.npy'))
hr_test = np.load(os.path.join(processed_data_path, 'hr_test.npy'))
rr_test = np.load(os.path.join(processed_data_path, 'rr_test.npy'))
print(f"Kích thước dữ liệu kiểm thử: {X_test.shape}")

# Chuẩn hóa dữ liệu kiểm thử
X_test = (X_test - np.min(X_test)) / (np.max(X_test) - np.min(X_test))
condition_test = tf.convert_to_tensor(np.column_stack((hr_test, rr_test)), dtype=tf.float32)
condition_test = (condition_test - tf.reduce_min(condition_test, axis=0)) / (tf.reduce_max(condition_test, axis=0) - tf.reduce_min(condition_test, axis=0))

# Tham số mô hình (phải khớp với Dense_CVAE_v3.6.py)
input_dim = X_test.shape[1]  # 1250
condition_dim = 2  # HR và RR
latent_dim = 20
hidden_units = [256, 128]
fs = 125  # Tần số lấy mẫu (Hz)

# Định nghĩa lớp Sampling
class Sampling(layers.Layer):
    def call(self, inputs):
        z_mean, z_log_var = inputs
        batch = tf.shape(z_mean)[0]
        dim = tf.shape(z_mean)[1]
        epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
        return z_mean + tf.exp(0.5 * z_log_var) * epsilon

# Định nghĩa mô hình CVAE (khớp với Dense_CVAE_v3.6.py)
class CVAE(tf.keras.Model):
    def __init__(self, input_dim, condition_dim, latent_dim, hidden_units):
        super(CVAE, self).__init__()
        self.input_dim = input_dim
        self.condition_dim = condition_dim
        self.latent_dim = latent_dim
        self.hidden_units = hidden_units

        # Xây dựng Encoder
        encoder_inputs = layers.Input(shape=(input_dim,), name='encoder_input')
        condition_inputs = layers.Input(shape=(condition_dim,), name='condition_input')
        x = layers.Concatenate()([encoder_inputs, condition_inputs])
        for units in hidden_units:
            x = layers.Dense(units)(x)
            x = layers.BatchNormalization()(x)
            x = layers.ReLU()(x)
        z_mean = layers.Dense(latent_dim, name='z_mean')(x)
        z_log_var = layers.Dense(latent_dim, name='z_log_var')(x)
        z = Sampling()([z_mean, z_log_var])
        self.encoder = Model([encoder_inputs, condition_inputs], [z_mean, z_log_var, z], name='encoder')

        # Xây dựng Decoder
        latent_inputs = layers.Input(shape=(latent_dim,), name='latent_input')
        condition_inputs_decoder = layers.Input(shape=(condition_dim,), name='condition_input')
        x = layers.Concatenate()([latent_inputs, condition_inputs_decoder])
        for units in reversed(hidden_units):
            x = layers.Dense(units)(x)
            x = layers.BatchNormalization()(x)
            x = layers.ReLU()(x)
        decoder_outputs = layers.Dense(input_dim, activation='sigmoid')(x)
        self.decoder = Model([latent_inputs, condition_inputs_decoder], decoder_outputs, name='decoder')

    def encode(self, inputs):
        return self.encoder(inputs)

    def decode(self, inputs):
        return self.decoder(inputs)

    def generate(self, condition, z=None):
        condition = tf.convert_to_tensor(condition, dtype=tf.float32)
        if z is None:
            batch_size = tf.shape(condition)[0]
            z = tf.random.normal(shape=(batch_size, self.latent_dim))
        return self.decode([z, condition])

# Tải mô hình đã huấn luyện
print("Đang tải mô hình CVAE đã huấn luyện...")
cvae = CVAE(input_dim, condition_dim, latent_dim, hidden_units)
cvae.build(input_shape=[(None, input_dim), (None, condition_dim)])
cvae.load_weights(os.path.join(model_path, 'cvae_dense_final_v3.weights.h5'))

# Tạo tín hiệu PPG từ mô hình
num_samples = 10
test_indices = np.random.choice(len(X_test), num_samples, replace=False)
test_conditions = tf.gather(condition_test, test_indices)
original_ppg = X_test[test_indices]
generated_ppg = cvae.generate(test_conditions).numpy()

# Hàm phân tích phổ tần số sử dụng FFT và trích xuất HR/RR
def analyze_frequency_spectrum(signal, fs):
    n = len(signal)
    yf = fft(signal)
    xf = fftfreq(n, 1/fs)[:n//2]
    yf_abs = 2.0/n * np.abs(yf[0:n//2])

    # Trích xuất HR (1-2 Hz ~ 60-120 bpm)
    hr_idx = (xf >= 1.0) & (xf <= 2.0)
    hr_freq = xf[hr_idx][np.argmax(yf_abs[hr_idx])] if np.any(hr_idx) else 0
    hr_est = hr_freq * 60  # Chuyển sang bpm

    # Trích xuất RR (0.1-0.5 Hz ~ 6-30 breaths/min)
    rr_idx = (xf >= 0.1) & (xf <= 0.5)
    rr_freq = xf[rr_idx][np.argmax(yf_abs[rr_idx])] if np.any(rr_idx) else 0
    rr_est = rr_freq * 60  # Chuyển sang breaths/min

    return xf, yf_abs, hr_est, rr_est

# Hàm phân tích phổ tần số sử dụng Welch's method
def analyze_welch_spectrum(signal, fs):
    f, Pxx = welch(signal, fs=fs, nperseg=min(256, len(signal)))
    return f, Pxx

# Hàm tìm đỉnh trong phổ tần số
def find_peaks_spectrum(xf, yf, threshold=0.1, min_distance=5):
    peaks, _ = find_peaks(yf, height=threshold*np.max(yf), distance=min_distance)
    return [(idx, xf[idx], yf[idx]) for idx in peaks]

# Hàm tính toán các chỉ số đánh giá
def calculate_metrics(original, generated):
    mse = mean_squared_error(original, generated)
    max_val = max(np.max(original), np.max(generated))
    psnr = 20 * np.log10(max_val / np.sqrt(mse)) if mse > 0 else float('inf')
    corr = np.corrcoef(original, generated)[0, 1]
    return mse, psnr, corr

# Hàm tính toán MSE trong miền tần số
def calculate_frequency_metrics(f_orig, psd_orig, f_gen, psd_gen):
    from scipy.interpolate import interp1d
    f_min = max(np.min(f_orig), np.min(f_gen))
    f_max = min(np.max(f_orig), np.max(f_gen))
    f_common = np.linspace(f_min, f_max, 1000)
    interp_orig = interp1d(f_orig, psd_orig, bounds_error=False, fill_value=0)
    interp_gen = interp1d(f_gen, psd_gen, bounds_error=False, fill_value=0)
    psd_orig_interp = interp_orig(f_common)
    psd_gen_interp = interp_gen(f_common)
    return mean_squared_error(psd_orig_interp, psd_gen_interp)

# Phân tích phổ tần số chi tiết
print("\nPhân tích phổ tần số chi tiết của tín hiệu PPG gốc và tín hiệu PPG đã tạo")
results_df = pd.DataFrame(columns=[
    'Sample', 'HR_Real', 'RR_Real', 'HR_Gen', 'RR_Gen', 'MSE_Time', 'PSNR', 'Corr', 'MSE_Freq',
    'Orig_Peak1_Freq', 'Orig_Peak2_Freq', 'Orig_Peak3_Freq',
    'Gen_Peak1_Freq', 'Gen_Peak2_Freq', 'Gen_Peak3_Freq'
])

for i in range(num_samples):
    print(f"\nPhân tích mẫu {i+1}:")

    # Phân tích tín hiệu gốc
    xf_orig, yf_orig, hr_orig_est, rr_orig_est = analyze_frequency_spectrum(original_ppg[i], fs)
    f_orig, psd_orig = analyze_welch_spectrum(original_ppg[i], fs)
    peaks_orig = find_peaks_spectrum(xf_orig, yf_orig)[:3]

    # Phân tích tín hiệu đã tạo
    xf_gen, yf_gen, hr_gen_est, rr_gen_est = analyze_frequency_spectrum(generated_ppg[i], fs)
    f_gen, psd_gen = analyze_welch_spectrum(generated_ppg[i], fs)
    peaks_gen = find_peaks_spectrum(xf_gen, yf_gen)[:3]

    # Tính toán các chỉ số
    mse_time, psnr, corr = calculate_metrics(original_ppg[i], generated_ppg[i])
    mse_freq = calculate_frequency_metrics(f_orig, psd_orig, f_gen, psd_gen)

    # Lấy các đỉnh tần số
    orig_peaks_freq = [p[1] for p in peaks_orig] + [0]*(3-len(peaks_orig))
    gen_peaks_freq = [p[1] for p in peaks_gen] + [0]*(3-len(peaks_gen))

    # In kết quả
    print(f"HR Real={hr_test[test_indices[i]]:.2f}, RR Real={rr_test[test_indices[i]]:.2f}")
    print(f"HR Gen={hr_gen_est:.2f}, RR Gen={rr_gen_est:.2f}")
    print(f"MSE (time): {mse_time:.4f}, PSNR: {psnr:.2f}dB, Corr: {corr:.4f}, MSE (freq): {mse_freq:.4f}")

    # Thêm vào DataFrame
    new_row = pd.DataFrame({
        'Sample': [i+1],
        'HR_Real': [hr_test[test_indices[i]]],
        'RR_Real': [rr_test[test_indices[i]]],
        'HR_Gen': [hr_gen_est],
        'RR_Gen': [rr_gen_est],
        'MSE_Time': [mse_time],
        'PSNR': [psnr],
        'Corr': [corr],
        'MSE_Freq': [mse_freq],
        'Orig_Peak1_Freq': [orig_peaks_freq[0]],
        'Orig_Peak2_Freq': [orig_peaks_freq[1]],
        'Orig_Peak3_Freq': [orig_peaks_freq[2]],
        'Gen_Peak1_Freq': [gen_peaks_freq[0]],
        'Gen_Peak2_Freq': [gen_peaks_freq[1]],
        'Gen_Peak3_Freq': [gen_peaks_freq[2]]
    })
    results_df = pd.concat([results_df, new_row], ignore_index=True)

    # Vẽ biểu đồ so sánh
    plt.figure(figsize=(15, 10))

    # Tín hiệu thời gian
    plt.subplot(2, 2, 1)
    plt.plot(original_ppg[i], label='Original PPG')
    plt.plot(generated_ppg[i], label='Generated PPG', alpha=0.7)
    plt.title(f'Sample {i+1}: Time Domain (HR={hr_test[test_indices[i]]:.1f}, RR={rr_test[test_indices[i]]:.1f})')
    plt.xlabel('Time')
    plt.ylabel('Amplitude')
    plt.legend()
    plt.grid(True, alpha=0.3)

    # Phổ FFT
    plt.subplot(2, 2, 2)
    plt.plot(xf_orig, yf_orig, label='Original FFT')
    plt.plot(xf_gen, yf_gen, label='Generated FFT', alpha=0.7)
    plt.title('FFT Spectrum')
    plt.xlabel('Frequency (Hz)')
    plt.ylabel('Amplitude')
    plt.xlim([0, 5])
    plt.legend()
    plt.grid(True, alpha=0.3)

    # Phổ PSD
    plt.subplot(2, 2, 3)
    plt.semilogy(f_orig, psd_orig, label='Original PSD')
    plt.semilogy(f_gen, psd_gen, label='Generated PSD', alpha=0.7)
    plt.title('Power Spectral Density (Welch)')
    plt.xlabel('Frequency (Hz)')
    plt.ylabel('PSD (dB/Hz)')
    plt.xlim([0, 5])
    plt.legend()
    plt.grid(True, alpha=0.3)

    # So sánh đỉnh
    plt.subplot(2, 2, 4)
    plt.scatter([p[1] for p in peaks_orig], [p[2] for p in peaks_orig], label='Orig Peaks', c='blue')
    plt.scatter([p[1] for p in peaks_gen], [p[2] for p in peaks_gen], label='Gen Peaks', c='orange', alpha=0.7)
    plt.title('Peak Frequencies')
    plt.xlabel('Frequency (Hz)')
    plt.ylabel('Amplitude')
    plt.xlim([0, 5])
    plt.legend()
    plt.grid(True, alpha=0.3)

    plt.tight_layout()
    plt.savefig(os.path.join(figures_path, f'sample_{i+1}_analysis.png'))
    plt.close()

# Lưu kết quả
results_df.to_csv(os.path.join(results_path, 'frequency_analysis_results_v2.csv'), index=False)
avg_metrics = results_df[['MSE_Time', 'PSNR', 'Corr', 'MSE_Freq']].mean()
print("\nKết quả trung bình:")
print(f"MSE (time): {avg_metrics['MSE_Time']:.4f}, PSNR: {avg_metrics['PSNR']:.2f}dB, Corr: {avg_metrics['Corr']:.4f}, MSE (freq): {avg_metrics['MSE_Freq']:.4f}")

# Lưu báo cáo
with open(os.path.join(results_path, 'fourier_analysis_results_v2.txt'), 'w') as f:
    f.write("PHÂN TÍCH BIẾN ĐỔI FOURIER (V2.2)\n")
    f.write("==============================\n\n")
    f.write(f"- Số mẫu: {num_samples}\n")
    f.write(f"- Kết quả trung bình: MSE (time): {avg_metrics['MSE_Time']:.4f}, PSNR: {avg_metrics['PSNR']:.2f}dB, Corr: {avg_metrics['Corr']:.4f}, MSE (freq): {avg_metrics['MSE_Freq']:.4f}\n\n")
    for i in range(len(results_df)):
        f.write(f"Mẫu {i+1}: HR Real={results_df['HR_Real'][i]:.2f}, RR Real={results_df['RR_Real'][i]:.2f}, HR Gen={results_df['HR_Gen'][i]:.2f}, RR Gen={results_df['RR_Gen'][i]:.2f}\n")
        f.write(f"  Metrics: MSE (time): {results_df['MSE_Time'][i]:.4f}, PSNR: {results_df['PSNR'][i]:.2f}dB, Corr: {results_df['Corr'][i]:.4f}, MSE (freq): {results_df['MSE_Freq'][i]:.4f}\n")
        f.write(f"  Peaks Orig: {results_df['Orig_Peak1_Freq'][i]:.2f}, {results_df['Orig_Peak2_Freq'][i]:.2f}, {results_df['Orig_Peak3_Freq'][i]:.2f} Hz\n")
        f.write(f"  Peaks Gen: {results_df['Gen_Peak1_Freq'][i]:.2f}, {results_df['Gen_Peak2_Freq'][i]:.2f}, {results_df['Gen_Peak3_Freq'][i]:.2f} Hz\n\n")

print(f"\nKết quả đã được lưu tại: {results_path}")

Đang tải dữ liệu kiểm thử...
Kích thước dữ liệu kiểm thử: (931, 1250)
Đang tải mô hình CVAE đã huấn luyện...

Phân tích phổ tần số chi tiết của tín hiệu PPG gốc và tín hiệu PPG đã tạo

Phân tích mẫu 1:
HR Real=0.31, RR Real=-0.86
HR Gen=84.00, RR Gen=12.00
MSE (time): 0.0111, PSNR: 17.02dB, Corr: -0.0943, MSE (freq): 0.0000


<ipython-input-39-7c8ffdfe9e4a>:210: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results_df = pd.concat([results_df, new_row], ignore_index=True)



Phân tích mẫu 2:
HR Real=-0.58, RR Real=-1.04
HR Gen=84.00, RR Gen=12.00
MSE (time): 0.0289, PSNR: 14.64dB, Corr: -0.0150, MSE (freq): 0.0000

Phân tích mẫu 3:
HR Real=0.50, RR Real=-0.55
HR Gen=84.00, RR Gen=12.00
MSE (time): 0.0705, PSNR: 10.86dB, Corr: 0.1030, MSE (freq): 0.0000

Phân tích mẫu 4:
HR Real=0.43, RR Real=0.52
HR Gen=84.00, RR Gen=12.00
MSE (time): 0.0070, PSNR: 17.85dB, Corr: 0.0747, MSE (freq): 0.0000

Phân tích mẫu 5:
HR Real=-1.56, RR Real=0.24
HR Gen=84.00, RR Gen=12.00
MSE (time): 0.0206, PSNR: 15.31dB, Corr: -0.0336, MSE (freq): 0.0000

Phân tích mẫu 6:
HR Real=-0.76, RR Real=-0.79
HR Gen=84.00, RR Gen=12.00
MSE (time): 0.0154, PSNR: 14.85dB, Corr: 0.0256, MSE (freq): 0.0000

Phân tích mẫu 7:
HR Real=-0.54, RR Real=0.22
HR Gen=84.00, RR Gen=12.00
MSE (time): 0.0189, PSNR: 16.21dB, Corr: 0.0974, MSE (freq): 0.0000

Phân tích mẫu 8:
HR Real=-0.39, RR Real=-0.18
HR Gen=84.00, RR Gen=12.00
MSE (time): 0.0121, PSNR: 16.63dB, Corr: -0.0281, MSE (freq): 0.0000

Phân tí

# **visualize_and_evaluate**

In [40]:
import numpy as np
import matplotlib.pyplot as plt
import os
import pandas as pd
from scipy.signal import welch, find_peaks
from scipy.fft import fft, fftfreq
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import tensorflow as tf
from tensorflow.keras import layers, Model
from tensorflow.keras.optimizers import Adam

# Đường dẫn đến dữ liệu và kết quả
processed_data_path = '/content/drive/MyDrive/bidmc/processed'
model_path = '/content/drive/MyDrive/bidmc/models'
figures_path = '/content/drive/MyDrive/bidmc/figures/visualize'
results_path = '/content/drive/MyDrive/bidmc/results'

# Tạo thư mục nếu chưa tồn tại
os.makedirs(figures_path, exist_ok=True)
os.makedirs(results_path, exist_ok=True)

# Tải dữ liệu kiểm thử và huấn luyện (để giải chuẩn hóa)
print("Đang tải dữ liệu kiểm thử và huấn luyện...")
X_test = np.load(os.path.join(processed_data_path, 'ppg_test.npy'))
hr_test = np.load(os.path.join(processed_data_path, 'hr_test.npy'))
rr_test = np.load(os.path.join(processed_data_path, 'rr_test.npy'))
X_train = np.load(os.path.join(processed_data_path, 'ppg_train.npy'))
hr_train = np.load(os.path.join(processed_data_path, 'hr_train.npy'))
rr_train = np.load(os.path.join(processed_data_path, 'rr_train.npy'))
print(f"Kích thước dữ liệu kiểm thử: {X_test.shape}")

# Chuẩn hóa dữ liệu kiểm thử
X_test = (X_test - np.min(X_test)) / (np.max(X_test) - np.min(X_test))
condition_test = tf.convert_to_tensor(np.column_stack((hr_test, rr_test)), dtype=tf.float32)
condition_test = (condition_test - tf.reduce_min(condition_test, axis=0)) / (tf.reduce_max(condition_test, axis=0) - tf.reduce_min(condition_test, axis=0))

# Giải chuẩn hóa HR và RR về giá trị thực tế
hr_min, hr_max = np.min(hr_train), np.max(hr_train)
rr_min, rr_max = np.min(rr_train), np.max(rr_train)
hr_test_real = hr_test * (hr_max - hr_min) + hr_min
rr_test_real = rr_test * (rr_max - rr_min) + rr_min

# Tham số mô hình
input_dim = X_test.shape[1]  # 1250
condition_dim = 2  # HR và RR
latent_dim = 20
hidden_units = [256, 128]
fs = 125  # Tần số lấy mẫu (Hz)

# Định nghĩa lớp Sampling
class Sampling(layers.Layer):
    def call(self, inputs):
        z_mean, z_log_var = inputs
        batch = tf.shape(z_mean)[0]
        dim = tf.shape(z_mean)[1]
        epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
        return z_mean + tf.exp(0.5 * z_log_var) * epsilon

# Định nghĩa mô hình CVAE (khớp với Dense_CVAE_v3.6.py)
class CVAE(tf.keras.Model):
    def __init__(self, input_dim, condition_dim, latent_dim, hidden_units):
        super(CVAE, self).__init__()
        self.input_dim = input_dim
        self.condition_dim = condition_dim
        self.latent_dim = latent_dim
        self.hidden_units = hidden_units

        # Encoder
        encoder_inputs = layers.Input(shape=(input_dim,), name='encoder_input')
        condition_inputs = layers.Input(shape=(condition_dim,), name='condition_input')
        x = layers.Concatenate()([encoder_inputs, condition_inputs])
        for units in hidden_units:
            x = layers.Dense(units)(x)
            x = layers.BatchNormalization()(x)
            x = layers.ReLU()(x)
        z_mean = layers.Dense(latent_dim, name='z_mean')(x)
        z_log_var = layers.Dense(latent_dim, name='z_log_var')(x)
        z = Sampling()([z_mean, z_log_var])
        self.encoder = Model([encoder_inputs, condition_inputs], [z_mean, z_log_var, z], name='encoder')

        # Decoder
        latent_inputs = layers.Input(shape=(latent_dim,), name='latent_input')
        condition_inputs_decoder = layers.Input(shape=(condition_dim,), name='condition_input')
        x = layers.Concatenate()([latent_inputs, condition_inputs_decoder])
        for units in reversed(hidden_units):
            x = layers.Dense(units)(x)
            x = layers.BatchNormalization()(x)
            x = layers.ReLU()(x)
        decoder_outputs = layers.Dense(input_dim, activation='sigmoid')(x)
        self.decoder = Model([latent_inputs, condition_inputs_decoder], decoder_outputs, name='decoder')

    def encode(self, inputs):
        return self.encoder(inputs)

    def decode(self, inputs):
        return self.decoder(inputs)

    def generate(self, condition, z=None):
        condition = tf.convert_to_tensor(condition, dtype=tf.float32)
        if z is None:
            batch_size = tf.shape(condition)[0]
            z = tf.random.normal(shape=(batch_size, self.latent_dim))
        return self.decode([z, condition])

# Tải mô hình đã huấn luyện
print("Đang tải mô hình CVAE đã huấn luyện...")
cvae = CVAE(input_dim, condition_dim, latent_dim, hidden_units)
cvae.build(input_shape=[(None, input_dim), (None, condition_dim)])
cvae.load_weights(os.path.join(model_path, 'cvae_dense_final_v3.weights.h5'))

# Tạo tín hiệu PPG và không gian tiềm ẩn
num_samples = 20
test_indices = np.random.choice(len(X_test), num_samples, replace=False)
test_conditions = tf.gather(condition_test, test_indices)
original_ppg = X_test[test_indices]
generated_ppg = cvae.generate(test_conditions).numpy()
# Chuyển đổi X_test[test_indices] thành tensor trước khi mã hóa
original_ppg_tensor = tf.convert_to_tensor(original_ppg, dtype=tf.float32)
z_mean, z_log_var, z = cvae.encode([original_ppg_tensor, test_conditions])

# Tải kết quả phân tích Fourier
print("Đang tải kết quả phân tích Fourier...")
fourier_results_path = os.path.join(results_path, 'frequency_analysis_results_v2.csv')
if os.path.exists(fourier_results_path):
    fourier_results = pd.read_csv(fourier_results_path)
    print(f"Đã tải kết quả phân tích Fourier: {len(fourier_results)} mẫu")
else:
    print("Không tìm thấy kết quả phân tích Fourier!")
    sys.exit(1)

# Hàm phân tích phổ tần số
def analyze_frequency_spectrum(signal, fs):
    n = len(signal)
    yf = fft(signal)
    xf = fftfreq(n, 1/fs)[:n//2]
    yf_abs = 2.0/n * np.abs(yf[0:n//2])
    return xf, yf_abs

# 1. Trực quan hóa tín hiệu PPG gốc và tổng hợp
print("\n1. Trực quan hóa tín hiệu PPG gốc và tín hiệu tổng hợp")
plt.figure(figsize=(15, 20))
for i in range(min(10, num_samples)):
    xf_orig, yf_orig = analyze_frequency_spectrum(original_ppg[i], fs)
    xf_gen, yf_gen = analyze_frequency_spectrum(generated_ppg[i], fs)

    plt.subplot(10, 2, 2*i+1)
    plt.plot(original_ppg[i], label='Original')
    plt.plot(generated_ppg[i], label='Generated', alpha=0.7)
    plt.title(f'Sample {i+1}: HR={hr_test_real[test_indices[i]]:.1f}, RR={rr_test_real[test_indices[i]]:.1f}')
    plt.xlabel('Sample')
    plt.ylabel('Amplitude')
    plt.legend()
    plt.grid(True, alpha=0.3)

    plt.subplot(10, 2, 2*i+2)
    plt.plot(xf_orig, yf_orig, label='Original FFT')
    plt.plot(xf_gen, yf_gen, label='Generated FFT', alpha=0.7)
    plt.title(f'FFT: Sample {i+1}')
    plt.xlabel('Frequency (Hz)')
    plt.ylabel('Amplitude')
    plt.xlim([0, 5])
    plt.legend()
    plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(os.path.join(figures_path, 'original_vs_generated_comparison.png'))
plt.close()

# 2. Trực quan hóa phân bố HR và RR thực tế
print("\n2. Trực quan hóa phân bố HR và RR")
plt.figure(figsize=(15, 5))
plt.subplot(1, 3, 1)
plt.scatter(hr_test_real, rr_test_real, alpha=0.5)
plt.title('HR vs RR Distribution (Real)')
plt.xlabel('HR (bpm)')
plt.ylabel('RR (breaths/min)')
plt.grid(True, alpha=0.3)

plt.subplot(1, 3, 2)
plt.hist(hr_test_real, bins=20, alpha=0.7)
plt.title('HR Distribution')
plt.xlabel('HR (bpm)')
plt.ylabel('Count')
plt.grid(True, alpha=0.3)

plt.subplot(1, 3, 3)
plt.hist(rr_test_real, bins=20, alpha=0.7)
plt.title('RR Distribution')
plt.xlabel('RR (breaths/min)')
plt.ylabel('Count')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(os.path.join(figures_path, 'hr_rr_distribution.png'))
plt.close()

# 3. Trực quan hóa không gian tiềm ẩn thực tế
print("\n3. Trực quan hóa không gian tiềm ẩn")
pca = PCA(n_components=2)
latent_2d = pca.fit_transform(z.numpy())
plt.figure(figsize=(15, 5))

plt.subplot(1, 2, 1)
scatter = plt.scatter(latent_2d[:, 0], latent_2d[:, 1], c=hr_test_real[test_indices], cmap='viridis', alpha=0.7)
plt.colorbar(scatter, label='HR (bpm)')
plt.title('Latent Space (PCA) - HR')
plt.xlabel('PC 1')
plt.ylabel('PC 2')
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
scatter = plt.scatter(latent_2d[:, 0], latent_2d[:, 1], c=rr_test_real[test_indices], cmap='plasma', alpha=0.7)
plt.colorbar(scatter, label='RR (breaths/min)')
plt.title('Latent Space (PCA) - RR')
plt.xlabel('PC 1')
plt.ylabel('PC 2')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(os.path.join(figures_path, 'latent_space_visualization.png'))
plt.close()

# 4. Trực quan hóa ảnh hưởng của HR và RR
print("\n4. Trực quan hóa ảnh hưởng của HR và RR")
hr_values = np.linspace(hr_min, hr_max, 5)
rr_values = np.linspace(rr_min, rr_max, 5)
hr_norm_values = (hr_values - hr_min) / (hr_max - hr_min)
rr_norm_values = (rr_values - rr_min) / (rr_max - rr_min)

plt.figure(figsize=(15, 15))
for i, hr in enumerate(hr_norm_values):
    for j, rr in enumerate(rr_norm_values):
        condition = np.array([[hr, rr]])
        ppg = cvae.generate(condition).numpy()[0]
        plt.subplot(5, 5, i*5+j+1)
        plt.plot(ppg)
        plt.title(f'HR={hr_values[i]:.1f}, RR={rr_values[j]:.1f}')
        plt.xlabel('Sample')
        plt.ylabel('Amplitude')
        plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(os.path.join(figures_path, 'hr_rr_effect_on_ppg.png'))
plt.close()

# 5. Trực quan hóa phổ tần số theo HR và RR
print("\n5. Trực quan hóa phổ tần số theo HR và RR")
plt.figure(figsize=(15, 10))
rr_fixed = (15 - rr_min) / (rr_max - rr_min)  # RR cố định ở 15 breaths/min
for i, hr in enumerate(hr_norm_values):
    condition = np.array([[hr, rr_fixed]])
    ppg = cvae.generate(condition).numpy()[0]
    xf, yf = analyze_frequency_spectrum(ppg, fs)
    plt.subplot(2, 3, i+1)
    plt.plot(xf, yf)
    plt.title(f'FFT: HR={hr_values[i]:.1f}, RR=15')
    plt.xlabel('Frequency (Hz)')
    plt.ylabel('Amplitude')
    plt.xlim([0, 5])
    plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(os.path.join(figures_path, 'hr_effect_on_frequency.png'))
plt.close()

plt.figure(figsize=(15, 10))
hr_fixed = (90 - hr_min) / (hr_max - hr_min)  # HR cố định ở 90 bpm
for i, rr in enumerate(rr_norm_values):
    condition = np.array([[hr_fixed, rr]])
    ppg = cvae.generate(condition).numpy()[0]
    xf, yf = analyze_frequency_spectrum(ppg, fs)
    plt.subplot(2, 3, i+1)
    plt.plot(xf, yf)
    plt.title(f'FFT: HR=90, RR={rr_values[i]:.1f}')
    plt.xlabel('Frequency (Hz)')
    plt.ylabel('Amplitude')
    plt.xlim([0, 5])
    plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(os.path.join(figures_path, 'rr_effect_on_frequency.png'))
plt.close()

# 6. Trực quan hóa kết quả đánh giá
print("\n6. Trực quan hóa kết quả đánh giá")
plt.figure(figsize=(15, 10))
plt.subplot(2, 2, 1)
plt.hist(fourier_results['MSE_Time'], bins=10, alpha=0.7)
plt.title('MSE (Time) Distribution')
plt.xlabel('MSE')
plt.ylabel('Count')
plt.grid(True, alpha=0.3)

plt.subplot(2, 2, 2)
plt.hist(fourier_results['PSNR'], bins=10, alpha=0.7)
plt.title('PSNR Distribution')
plt.xlabel('PSNR (dB)')
plt.ylabel('Count')
plt.grid(True, alpha=0.3)

plt.subplot(2, 2, 3)
plt.hist(fourier_results['Corr'], bins=10, alpha=0.7)
plt.title('Correlation Distribution')
plt.xlabel('Correlation')
plt.ylabel('Count')
plt.grid(True, alpha=0.3)

plt.subplot(2, 2, 4)
plt.scatter(fourier_results['HR_Real'], fourier_results['HR_Gen'] - fourier_results['HR_Real'], label='HR Error')
plt.scatter(fourier_results['RR_Real'], fourier_results['RR_Gen'] - fourier_results['RR_Real'], label='RR Error', alpha=0.7)
plt.title('HR/RR Reconstruction Error')
plt.xlabel('Real Value')
plt.ylabel('Error')
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(os.path.join(figures_path, 'evaluation_metrics_distribution.png'))
plt.close()

# 7. Tạo bảng tóm tắt kết quả đánh giá
print("\n7. Tạo bảng tóm tắt kết quả đánh giá")
summary_stats = {
    'MSE_Time': fourier_results['MSE_Time'].describe().to_dict(),
    'PSNR': fourier_results['PSNR'].describe().to_dict(),
    'Corr': fourier_results['Corr'].describe().to_dict(),
    'MSE_Freq': fourier_results['MSE_Freq'].describe().to_dict(),
    'HR_Error': (fourier_results['HR_Gen'] - fourier_results['HR_Real']).describe().to_dict(),
    'RR_Error': (fourier_results['RR_Gen'] - fourier_results['RR_Real']).describe().to_dict()
}
summary_df = pd.DataFrame.from_dict(summary_stats, orient='index')
summary_df.to_csv(os.path.join(results_path, 'evaluation_summary_v2.csv'))

# Lưu báo cáo
with open(os.path.join(results_path, 'model_evaluation_results_v2.txt'), 'w') as f:
    f.write("KẾT QUẢ ĐÁNH GIÁ MÔ HÌNH CVAE (V2.1)\n")
    f.write("===================================\n\n")
    f.write(f"MSE (Time): Mean={summary_stats['MSE_Time']['mean']:.4f}, Std={summary_stats['MSE_Time']['std']:.4f}\n")
    f.write(f"PSNR: Mean={summary_stats['PSNR']['mean']:.2f}, Std={summary_stats['PSNR']['std']:.2f}\n")
    f.write(f"Corr: Mean={summary_stats['Corr']['mean']:.4f}, Std={summary_stats['Corr']['std']:.4f}\n")
    f.write(f"MSE (Freq): Mean={summary_stats['MSE_Freq']['mean']:.4f}, Std={summary_stats['MSE_Freq']['std']:.4f}\n")
    f.write(f"HR Error: Mean={summary_stats['HR_Error']['mean']:.2f}, Std={summary_stats['HR_Error']['std']:.2f}\n")
    f.write(f"RR Error: Mean={summary_stats['RR_Error']['mean']:.2f}, Std={summary_stats['RR_Error']['std']:.2f}\n\n")
    f.write("Nhận xét: Mô hình tái tạo tín hiệu PPG với chất lượng trung bình, nhưng chưa phản ánh tốt HR/RR thực tế.\n")

print(f"Kết quả đã được lưu tại: {results_path}")

Đang tải dữ liệu kiểm thử và huấn luyện...
Kích thước dữ liệu kiểm thử: (931, 1250)
Đang tải mô hình CVAE đã huấn luyện...
Đang tải kết quả phân tích Fourier...
Đã tải kết quả phân tích Fourier: 10 mẫu

1. Trực quan hóa tín hiệu PPG gốc và tín hiệu tổng hợp

2. Trực quan hóa phân bố HR và RR

3. Trực quan hóa không gian tiềm ẩn

4. Trực quan hóa ảnh hưởng của HR và RR

5. Trực quan hóa phổ tần số theo HR và RR

6. Trực quan hóa kết quả đánh giá

7. Tạo bảng tóm tắt kết quả đánh giá
Kết quả đã được lưu tại: /content/drive/MyDrive/bidmc/results
